In [ ]:
!export TOKENIZERS_PARALLELISM=false
!python -m pip install --upgrade pip
!pip install typing_extensions pydantic openai
!pip install datasets transformers peft trl bitsandbytes
!pip install sentence-transformers langchain langchain_community
!pip install chromadb
#!pip uninstall numpy -y
# !pip install numpy==1.26.4
!pip install "numpy<2"

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager
from langsmith import Client
import os
import uuid
import time
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pipeline 래핑
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1,
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 임베딩 및 벡터스토어 로드
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vector_store = Chroma(
    persist_directory="chroma_index",
    embedding_function=embedding
)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.7
    }
)

# ✅ 사용자 정의 프롬프트
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        너는 항공기 반입 금지 및 제한 물품에 대하여 친절하게 답해주고 나라별 여행지 추천 및 문화차이를 잘 설명할 수 있어
        문맥에 해당 하는 내용들만 신뢰도있게 대답해야해

        ### 질문:
        {question}
        
        ### 문맥:
        {context}

        ### 답변:
    """
)

In [ ]:
import json
from langchain.chains import RetrievalQA

# JSON 파일 로드
with open("test_case.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

test_cases = test_data["test_cases"]

# RetrievalQA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

# 테스트 실행
for test in test_cases:
    print(f"\n[테스트 ID: {test['id']}]")
    print(f"질문: {test['question']}")
    expected = test["expected_answer"]
    
    # 모델 응답 얻기
    result = qa_chain.run(test["question"])
    
    print(f"✅ 기대 응답: {expected}")
    print(f"🤖 모델 응답: {result["result"].split("### 답변:\n")[-1].strip()}")
